In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for data in dataloders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                #running_loss += loss.data[0]
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data).to(torch.float32)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

if __name__ == '__main__':

    # data_transform, pay attention that the input of Normalize() is Tensor and the input of RandomResizedCrop() or RandomHorizontalFlip() is PIL Image
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Scale(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # your image data file
    #data_dir = 'E:\design\data'
    image_datasets = {x: datasets.ImageFolder('E:\design\data',
                                              data_transforms[x]) for x in ['train', 'val']}
    # wrap your data and label into Tensor
    dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                 batch_size=4,
                                                 shuffle=True,
                                                 num_workers=4) for x in ['train', 'val']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

    # use gpu or not
    use_gpu = torch.cuda.is_available()

    # get model and replace the original fc layer with your fc layer
    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)

    if use_gpu:
        model_ft = model_ft.cuda()

    # define loss function
    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    model_ft = train_model(model=model_ft,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=exp_lr_scheduler,
                           num_epochs=25)

D:\Anacoda\lib\site-packages\torchvision\transforms\transforms.py:618: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +
D:\Anacoda\lib\site-packages\torchvision\transforms\transforms.py:207: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


Epoch 0/24
----------
train Loss: 0.1144 Acc: 0.8002
val Loss: 0.0578 Acc: 0.8971
Epoch 1/24
----------
train Loss: 0.1028 Acc: 0.8298
val Loss: 0.0487 Acc: 0.9182
Epoch 2/24
----------
train Loss: 0.1128 Acc: 0.8318
val Loss: 0.0578 Acc: 0.8946
Epoch 3/24
----------
train Loss: 0.0902 Acc: 0.8539
val Loss: 0.0529 Acc: 0.9217
Epoch 4/24
----------
train Loss: 0.0844 Acc: 0.8534
val Loss: 0.0450 Acc: 0.9357
Epoch 5/24
----------
train Loss: 0.0865 Acc: 0.8760
val Loss: 0.0528 Acc: 0.9222
Epoch 6/24
----------
train Loss: 0.0813 Acc: 0.8660
val Loss: 0.0385 Acc: 0.9493
Epoch 7/24
----------
train Loss: 0.0646 Acc: 0.9031
val Loss: 0.0403 Acc: 0.9347
Epoch 8/24
----------
train Loss: 0.0592 Acc: 0.9106
val Loss: 0.0368 Acc: 0.9493
Epoch 9/24
----------
train Loss: 0.0603 Acc: 0.9086
val Loss: 0.0350 Acc: 0.9458
Epoch 10/24
----------
train Loss: 0.0559 Acc: 0.9192
val Loss: 0.0334 Acc: 0.9503
Epoch 11/24
----------
train Loss: 0.0554 Acc: 0.9121
val Loss: 0.0296 Acc: 0.9553
Epoch 12/24
--

In [3]:
torch.save(model_ft, 'model.pkl')
#model = torch.load('model.pkl')

In [3]:
model = torch.load('model.pkl')

D:\anaconda\lib\site-packages\torch\serialization.py:454: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
D:\anaconda\lib\site-packages\torch\serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
D:\anaconda\lib\site-packages\torch\cuda\__init__.py:118: UserWarning: 
    Found GPU0 GeForce GT 740 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.wa

In [2]:
import torch

In [4]:
print(type(model))

<class 'torchvision.models.resnet.ResNet'>


In [10]:
from PIL import Image
from io import BytesIO
image = Image.open('0e96ddb45429ee0c81d55bb33f500894.jpg')

In [15]:
res = model.eval()
import numpy as np


In [20]:
from torch.autograd import Variable

In [27]:
imagelist = list(image.getdata())

In [31]:
npImage=np.array(imagelist)

In [32]:
res(npImage)

AttributeError: 'Conv2d' object has no attribute 'padding_mode'